# Covid-19 - India Reports
 # 2020 coronavirus pandemic in India
The first case of the 2019–20 coronavirus pandemic in India was reported on 30 January 2020, originating from China. As of 26 March 2020, the Indian Council of Medical Research and Ministry of Health and Family Welfare have confirmed a total of 646 cases, 42 recoveries, 1 migration and 10 deaths in the country. The infection rate of COVID-19 in India is reported to be 1.7, which is remarkably lower than in the worst affected countries.There are concerns that testing for the virus in India is being inadequately conducted and the need for ramping up the scale of testing is a necessity to understand the real scope of the number of people affected.

* Import libraries and initial setup:**

In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex2 import *
print("Setup Complete")

/kaggle/input/covid19-in-india/HospitalBedsIndia.csv
/kaggle/input/covid19-in-india/IndividualDetails.csv
/kaggle/input/covid19-in-india/population_india_census2011.csv
/kaggle/input/covid19-in-india/covid_19_india.csv
/kaggle/input/covid19-in-india/ICMRTestingDetails.csv
Setup Complete


import futher libraries

In [2]:
import pandas as pd
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/covid19-in-india/HospitalBedsIndia.csv
/kaggle/input/covid19-in-india/IndividualDetails.csv
/kaggle/input/covid19-in-india/population_india_census2011.csv
/kaggle/input/covid19-in-india/covid_19_india.csv
/kaggle/input/covid19-in-india/ICMRTestingDetails.csv


In [3]:
covid_19_India = pd.read_csv("../input/covid19-in-india/covid_19_india.csv")
IndividualDetails = pd.read_csv("../input/covid19-in-india/IndividualDetails.csv")
HospitalBedsIndia = pd.read_csv("../input/covid19-in-india/HospitalBedsIndia.csv")
ICMRTestingDetails = pd.read_csv("../input/covid19-in-india/ICMRTestingDetails.csv")
population_india_census2011 = pd.read_csv("../input/covid19-in-india/population_india_census2011.csv")

covid_19_India.columns

Index(['Sno', 'Date', 'State/UnionTerritory', 'ConfirmedIndianNational',
       'ConfirmedForeignNational', 'Cured', 'Deaths'],
      dtype='object')

In [4]:
features=['Sno', 'Date', 'State/UnionTerritory', 'ConfirmedIndianNational',
       'ConfirmedForeignNational', 'Cured', 'Deaths']
covid_19_India=covid_19_India[features]
display(covid_19_India)

,Sno,Date,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths
0,1,30/01/20,Kerala,1,0,0,0
1,2,31/01/20,Kerala,1,0,0,0
2,3,01/02/20,Kerala,2,0,0,0
3,4,02/02/20,Kerala,3,0,0,0
4,5,03/02/20,Kerala,3,0,0,0
...,...,...,...,...,...,...,...
386,387,26/03/20,Jammu and Kashmir,11,0,1,0
387,388,26/03/20,Ladakh,13,0,0,0
388,389,26/03/20,Uttar Pradesh,36,1,11,0
389,390,26/03/20,Uttarakhand,4,1,0,0


In [5]:
import numpy as np 
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import init_notebook_mode, plot, iplot, download_plotlyjs
import plotly as ply
import pycountry
import folium 
from folium import plugins
import json


%config InlineBackend.figure_format = 'retina'
init_notebook_mode(connected=True)

# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow

# India Latitude Longitude
India_Latitude = 21.7679
India_Longitude = 78.8718 

In [6]:
# Utility Functions

'''Display markdown formatted output like bold, italic bold etc.'''
def formatted_text(string):
    display(Markdown(string))


'''highlight the maximum in a Series or DataFrame'''  
def highlight_max(data, color='red'):
    attr = 'background-color: {}'.format(color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''), index=data.index, columns=data.columns)
    


# Utility Plotting Functions

def plotDailyReportedCasesOverTime(df, country):
    # confirmed
    fig = px.bar(df, x="Date", y="Confirmed")
    layout = go.Layout(
 title=go.layout.Title(text="Daily count of confirmed cases in "+ country, x=0.5),
        font=dict(size=14),
        width=800,
        height=500,
        xaxis_title = "Date",
        yaxis_title = "Confirmed cases")

    fig.update_layout(layout)
    fig.show()

    # deaths
    fig = px.bar(df, x="Date", y="Deaths")
    layout = go.Layout(
        title=go.layout.Title(text="Daily count of reported deaths in "+ country, x=0.5),
        font=dict(size=14),
        width=800,
        height=500,
        xaxis_title = "Date",
        yaxis_title = "Deaths Reported")

    fig.update_layout(layout)
    fig.show()

    # recovered
    fig = px.bar(df, x="Date", y="Recovered")
    layout = go.Layout(
        title=go.layout.Title(text="Daily count of recovered cases in "+ country, x=0.5),
        font=dict(size=14),
        width=800,
        height=500,
        xaxis_title = "Date",
        yaxis_title = "Recovered Cases")

    fig.update_layout(layout)
    fig.show()
    
# Cases over time
def scatterPlotCasesOverTime(df, country):
    plot = make_subplots(rows=1, cols=3, subplot_titles=("Comfirmed", "Deaths", "Recovered"))

    subPlot1 = go.Scatter(
                    x=df['Date'],
                    y=df['Confirmed'],
                    name="Confirmed",
                    line_color='orange',
        opacity=0.8)

    subPlot2 = go.Scatter(
                    x=df['Date'],
                    y=df['Deaths'],
                    name="Deaths",
                    line_color='red',
                    opacity=0.8)

    subPlot3 = go.Scatter(
                    x=df['Date'],
                    y=df['Recovered'],
                    name="Recovered",
                    line_color='green',
                    opacity=0.8)

    plot.append_trace(subPlot1, 1, 1)
    plot.append_trace(subPlot2, 1, 2)
    plot.append_trace(subPlot3, 1, 3)
    plot.update_layout(template="ggplot2", title_text = country + '<b> - Spread of the nCov Over Time</b>')

    plot.show()

In [7]:
covid_19_India['Confirmed'] = covid_19_India['ConfirmedIndianNational'] + covid_19_India['ConfirmedForeignNational']
covid_19_India.rename(columns={'State/UnionTerritory': 'State'}, inplace=True)
display(covid_19_India)

,Sno,Date,State,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,30/01/20,Kerala,1,0,0,0,1
1,2,31/01/20,Kerala,1,0,0,0,1
2,3,01/02/20,Kerala,2,0,0,0,2
3,4,02/02/20,Kerala,3,0,0,0,3
4,5,03/02/20,Kerala,3,0,0,0,3
...,...,...,...,...,...,...,...,...
386,387,26/03/20,Jammu and Kashmir,11,0,1,0,11
387,388,26/03/20,Ladakh,13,0,0,0,13
388,389,26/03/20,Uttar Pradesh,36,1,11,0,37
389,390,26/03/20,Uttarakhand,4,1,0,0,5


# State wise bar diagram of the confirmed cases(indian + foriegn cases):

In [8]:
statewise_cases = pd.DataFrame(covid_19_India.groupby(['State'])['Confirmed', 'Deaths', 'Cured'].max().reset_index())
statewise_cases["Country"] = "India" # in order to have a single root node
fig = px.bar(statewise_cases,x="Confirmed", y="State", title='Confirmed Cases', text='Confirmed', orientation='h', 
             width=16*(max(statewise_cases['Confirmed']) + 2), height=700, range_x = [0, max(statewise_cases['Confirmed']) + 2])
fig.update_traces(marker_color='#0726ed', opacity=0.8, textposition='outside')

fig.update_layout(plot_bgcolor='rgb(208, 236, 245)')
fig.show()

# Date wise analysis

In [9]:
date_wise_data = covid_19_India[["Date","Confirmed","Deaths","Cured"]]
date_wise_data['Date'] = date_wise_data['Date'].apply(pd.to_datetime, dayfirst=True)
date_wise_data

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Confirmed,Deaths,Cured
0,2020-01-30,1,0,0
1,2020-01-31,1,0,0
2,2020-02-01,2,0,0
3,2020-02-02,3,0,0
4,2020-02-03,3,0,0
...,...,...,...,...
386,2020-03-26,11,0,1
387,2020-03-26,13,0,0
388,2020-03-26,37,0,11
389,2020-03-26,5,0,0


In [10]:
date_wise_data = date_wise_data.groupby(["Date"]).sum().reset_index()

formatted_text('***Date wise data***')
date_wise_data

***Date wise data***

,Date,Confirmed,Deaths,Cured
0,2020-01-30,1,0,0
1,2020-01-31,1,0,0
2,2020-02-01,2,0,0
3,2020-02-02,3,0,0
4,2020-02-03,3,0,0
5,2020-02-04,3,0,0
6,2020-02-05,3,0,0
7,2020-02-06,3,0,0
8,2020-02-07,3,0,0
9,2020-02-08,3,0,0


In [11]:
temp = date_wise_data.melt(id_vars="Date", value_vars=['Cured', 'Deaths', 'Confirmed'],
                 var_name='Case', value_name='Count')

fig = px.area(temp, x="Date", y="Count", color='Case',title='Time wise cases analysis', color_discrete_sequence = [rec, dth, act])
fig.show()



# State wise analysis(Tree map)

In [12]:
statewise_cases = pd.DataFrame(covid_19_India.groupby(['State'])['Confirmed', 'Deaths', 'Cured'].max().reset_index())
statewise_cases["Country"] = "India" # in order to have a single root node
fig = px.treemap(statewise_cases, path=['Country','State'], values='Confirmed',
                  color='Confirmed', hover_data=['State'],
                  color_continuous_scale='RdBu')
fig.show()